In [ ]:
# Importing required libraries
from tkinter import *
from tkinter import ttk
from tkinter import messagebox as ms
from PIL import ImageTk, Image
import time
import os
import cv2
import smtplib
import numpy as np
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import keras
import tensorflow
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
    
# Ignore the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Define colors and fonts
white = "#ffffff"
lightBlue2 = "#adc5ed"
font = "Constantia"
fontButtons = (font, 12)
maxWidth = 1200
maxHeight= 600
a=1

def popup_window(): # create a tkinter popup window with a progress bar.
  popuproot=Tk() # Create a tkinter root window
  popuproot.title('FACE MASK')
  popuproot.iconbitmap('mask icon.ico')
  popuproot.resizable(False,False)

  # Progress bar widget
  progress = ttk.Progressbar(popuproot, orient=HORIZONTAL, length=1000, mode='determinate')  
    
    
  # Create a label for the progress bar   
  lb1 = Label(popuproot, text='', fg='black', width=25, height=1, font=('calibri', 25)) 

  def bar(): # Update the progress bar and call the TrackImages function to detect face masks and send email alerts.
        
    lb1.config(text="starting app")    # Update progress bar text and value
    progress['value'] = 20
    popuproot.update_idletasks()

    time.sleep(1)
    
    lb1.config(text="Loading basic packages")    # Update progress bar text and value
    progress['value'] = 40
    popuproot.update_idletasks()


    time.sleep(1)
    
    lb1.config(text="Loading Deep learning packages")    # Update progress bar text and value
    progress['value'] = 50
    popuproot.update_idletasks()

     
    time.sleep(1)
    
    lb1.config(text="Loading trained model")    # Update progress bar text and value
    progress['value'] = 60
    popuproot.update_idletasks()

    model = load_model('model2.h5')    # Load the pre-trained deep learning model

    time.sleep(1)
    
    lb1.config(text="loading the Application")    # Update progress bar text and value
    progress['value'] = 80
    popuproot.update_idletasks()

    def send_an_email(file_path): # send an email with an image attachment when a person is detected without a mask.     
      email_user = 'aeroapj@gmail.com'       # Define email credentials and subject  
      email_password = 'demdum*&987'
      email_send = "aeroapj@gmail.com"
      subject = 'No mask detected'      
      msg = MIMEMultipart()       # Create a MIME object
      msg['From'] = email_user
      msg['To'] = email_send
      msg['Subject'] = subject

      body = 'Absence of mask detected!'       # Add body to the email
      msg.attach(MIMEText(body, 'plain'))

      filename = file_path       # Attach the image to the email
      attachment = open(filename, 'rb')
      part = MIMEBase('application', 'octet-stream')
      part.set_payload((attachment).read())
      encoders.encode_base64(part)
      part.add_header('Content-Disposition', "attachment; filename= " + filename)
      msg.attach(part)
      text = msg.as_string()       # Convert the MIME object to string
      server = smtplib.SMTP('smtp.gmail.com', 587)       # Send the email
      server.starttls()
      server.login(email_user, email_password)
      server.sendmail(email_user, email_send, text)
      server.quit()
    
    def TrackImages(): 
      # Detect face masks in a live video feed and send an email alert when a person is detected without a mask.
      harcascadePath = "haarcascade_frontalface_default.xml"       # Load the face cascade
      faceCascade = cv2.CascadeClassifier(harcascadePath)
    
      # Create a new tkinter window  
      mainWindow = Toplevel(popuproot)
      mainWindow.title("face mask Management")
      mainWindow.iconbitmap('mask icon.ico')
      mainWindow.configure(bg='white')
      mainWindow.geometry('%dx%d+%d+%d' % (maxWidth, maxHeight, 0, 0))
    
      # Set row and column weights
      mainWindow.grid_rowconfigure(0, weight=1)       
      mainWindow.grid_columnconfigure(0, weight=1)
    
      # Create the main frame  
      mainFrame = Frame(mainWindow)  
      
    
      mainFrame.place(x=20, y=20)
      lmain = Label(mainFrame)       
    
      # Create a label for the video feed
      lmain.pack(side=LEFT)
      cam = cv2.VideoCapture(0)       # Open the camera
      font = cv2.FONT_HERSHEY_SIMPLEX
    
      def show_frame(): # Show the video feed in the tkinter window, detects face masks, and updates the GUI.    
          global a
          # Read a frame from the camera
          ret, im = cam.read()           
          if im.size > 0:  
            gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)             # Convert the frame to grayscale
            faces = faceCascade.detectMultiScale(gray, 1.2, 5)             # Detect faces in the frame
            
            for (x, y, w, h) in faces:
                
                rol_color = im[y - 20:y + h + 20, x - 20:x + w + 20]                 # Extract the face from the frame
                print("scanning for no mask")
                print("Checking for non-wearing mask")
                cv2.imwrite("mask_check/" + str(a) + ".jpg", rol_color)                # Save the face as an image
                X = cv2.imread("mask_check/" + str(a) + ".jpg", 3)  # Pre-process the image for the deep learning model
                X = cv2.resize(X, (150, 150))
                X = np.array(X)
                X = np.expand_dims(X, axis=0)
                face = model.predict(X)                # Predict the presence of a mask
                faceperval=round(face[1]*100,2)# Calculate the percentage of no mask detection
                if faceperval > 80:
                    cv2.rectangle(im, (x - 20, y - 20), (x + w + 20, y + h + 20), (0, 0, 255), 2) # Draw a red rectangle and text on the frame if no mask is detected
                    cv2.putText(im, str("No Mask"), (x - 20, y + h + 20), font, 1, (0, 0, 255), 2)
                    print("Absence of mask detected")
                    print("send text alert to email")
                    img2 = Image.open("mask_check/" + str(a) + ".jpg", 'r') # Show the detected face without a mask in the GUI
                    img2 = img2.resize((300, 300), Image.ANTIALIAS)
                    img2 = ImageTk.PhotoImage(img2)
                    panel = Label(mainWindow, image=img2)
                    panel.image = img2
                    panel.place(x=800, y=100)              
                    try:
                      send_an_email("mask_check/" + str(a) + ".jpg")                       # Send an email alert
                      print("mail sent")
                        
                    except:
                      print("issue in mail send")

                    faceper = "It's {} no face mask detected".format(faceperval) # Update the GUI with the no mask percentage
                    lbl2.config(text=faceper)
                    txt3.config(text="mail has been forwarded")
                    a += 1
                else:
                    cv2.rectangle(im, (x - 20, y - 20), (x + w + 20, y + h + 20), (255, 255, 255), 2) # Draw a white rectangle and text on the frame if a mask is detected
                    cv2.putText(im, str("detected face"), (x - 20, y + h + 20), font, 1, (255, 255, 255), 2)                    
                    
            cv2image = cv2.cvtColor(im, cv2.COLOR_BGR2RGBA)             # Convert the frame to RGBA and resize it
            img = Image.fromarray(cv2image).resize((650, 450))
            imgtk = ImageTk.PhotoImage(image=img)
            lmain.imgtk = imgtk             # Update the video feed label with the new frame
            lmain.configure(image=imgtk)

          lmain.after(10, show_frame)          # Repeat after 10 ms

      # Create a close button
      closeButton = Button(mainWindow, text="close", font=('calibri', 18, 'bold'), bg="lightblue", activebackground="red", activeforeground="white", width=20)      
      closeButton.configure(command=lambda: [cam.release(), mainWindow.destroy()])
      closeButton.place(x=230, y=500)
        
      # Add more labels to the GUI
      ldl1 = Label(mainWindow, text="Face mask detection", height=2, fg="black", bg="lightblue", font=('calibri', 15, 'bold'))       
      ldl1.place(x=850, y=20)
        
      lb = Label(mainWindow, text='', bg='lightblue', width=50, height=23)
      lb.place(x=780, y=80)

      lbl2 = Label(mainWindow, text="", width=35, fg="black", bg="lightblue", height=2,font=('calibri', 15, 'bold'))
      lbl2.place(x=780, y=450)
    
      lbl3 = Label(mainWindow, text="Mail: ", width=13, fg="black", bg="lightblue", height=2, font=('calibri', 15, 'bold'))
      lbl3.place(x=620, y=520)

      txt3 = Label(mainWindow, text="", bg="lightblue", fg="black", width=35, height=2, activebackground="lightblue", font=('calibri', 15, 'bold'))
      txt3.place(x=780, y=520)
    
      show_frame()      # Initialize the video feed   
      # Start the main loop of the GUI
      mainWindow.mainloop()      
        
    progress['value'] = 80      # Update the progress bar
    popuproot.update_idletasks()
    time.sleep(1)
    lb1.config(text="process completed")
    progress['value'] = 100
    TrackImages()      # Call the TrackImages function
         
  progress.pack(side=TOP,padx=21,pady = 10)  # Initialize the progress bar
  lb1.pack(side=TOP,pady = 10) 
    
    
  Button(popuproot, text = 'Start',width = 10,height = 1,fg="black"  ,bg="lightblue" ,font=('calibri', 15, ' bold '), command = bar).pack(side=BOTTOM,pady = 10)   # Create a start button
  popuproot.mainloop()  # Start the main loop of the popup window

popup_window() # Call the main function to start the application